In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd 
import gensim
import numpy as np
import pandas as pd
import os
import re
import csv
import codecs
import sys
import pickle
from csv import DictReader

# from utils.dataset import DataSet
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding,LSTM, Dense
from keras.models import Sequential
import nltk

nltk.download('punkt')
nltk.download('wordnet')



from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from numpy import zeros

from keras import callbacks
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, BatchNormalization, Activation, Bidirectional,Flatten
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences

import matplotlib as mpl
%matplotlib inline
from matplotlib import pyplot as plt
from keras.utils import plot_model 
from IPython.display import Image
# import pydot
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import tensorflow as tf

from numpy import asarray
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from string import punctuation

from keras import backend as K
from keras.engine.topology import Layer
#from keras import initializations
from keras import initializers, regularizers, constraints

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
#Adapted from https://github.com/FakeNewsChallenge/fnc-1/blob/master/scorer.py
#Original credit - @bgalbraith

LABELS = ['agree', 'disagree', 'discuss', 'unrelated']
LABELS_RELATED = ['unrelated','related']
RELATED = LABELS[0:3]

def score_submission(gold_labels, test_labels):
    score = 0.0
    cm = [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]]

    for i, (g, t) in enumerate(zip(gold_labels, test_labels)):
        g_stance, t_stance = g, t
        if g_stance == t_stance:
            score += 0.25
            if g_stance != 'unrelated':
                score += 0.50
        if g_stance in RELATED and t_stance in RELATED:
            score += 0.25

        cm[LABELS.index(g_stance)][LABELS.index(t_stance)] += 1

    return score, cm


def print_confusion_matrix(cm):
    lines = []
    header = "|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format('', *LABELS)
    line_len = len(header)
    lines.append("-"*line_len)
    lines.append(header)
    lines.append("-"*line_len)

    hit = 0
    total = 0
    for i, row in enumerate(cm):
        hit += row[i]
        total += sum(row)
        lines.append("|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format(LABELS[i],
                                                                   *row))
        lines.append("-"*line_len)
    print('\n'.join(lines))


def report_score(actual,predicted):
    score,cm = score_submission(actual,predicted)
    print("Score",score)
    best_score, _ = score_submission(actual,actual)
    # print("best_Score",best_score)

    print_confusion_matrix(cm)
    print("Score: " +str(score) + " out of " + str(best_score) + "\t("+str(score*100/best_score) + "%)")
    return score*100/best_score


if __name__ == "__main__":
    actual = [0,0,0,0,1,1,0,3,3]
    predicted = [0,0,0,0,1,1,2,3,3]

    report_score([LABELS[e] for e in actual],[LABELS[e] for e in predicted])

Score 6.75
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |     4     |     0     |     1     |     0     |
-------------------------------------------------------------
| disagree  |     0     |     2     |     0     |     0     |
-------------------------------------------------------------
|  discuss  |     0     |     0     |     0     |     0     |
-------------------------------------------------------------
| unrelated |     0     |     0     |     0     |     2     |
-------------------------------------------------------------
Score: 6.75 out of 7.5	(90.0%)


In [ ]:
class DataSet():
    def __init__(self, name="train", path="./data"):
        self.path = path

        print("Reading dataset")
        bodies = name+"_bodies.csv"
        stances = name+"_stances.csv"

        self.stances = self.read(stances)
        articles = self.read(bodies)
        self.articles = dict()

        #make the body ID an integer value
        for s in self.stances:
            s['Body ID'] = int(s['Body ID'])

        #copy all bodies into a dictionary
        for article in articles:
            self.articles[int(article['Body ID'])] = article['articleBody']

        print("Total stances: " + str(len(self.stances)))
        print("Total bodies: " + str(len(self.articles)))



    def read(self,filename):
        rows = []
        with open(self.path + "/" + filename, "r", encoding='utf-8') as table:
            r = DictReader(table)

            for line in r:
                rows.append(line)
        return rows


In [ ]:
def generate_features(stances,dataset):
    h, b, y = [],[],[]
    LABELS = ['agree', 'disagree', 'discuss', 'unrelated']
    for stance in stances:

        # print(stance['Stance'])
        y.append(LABELS.index(stance['Stance']))
        h.append(stance['Headline'])
        b.append(dataset.articles[stance['Body ID']])
   

    return(h,b,y)


In [ ]:
import sys
import numpy as np

# from sklearn.ensemble import GradientBoostingClassifier
# from feature_engineering import refuting_features, polarity_features, hand_features, gen_or_load_feats
# from feature_engineering import word_overlap_features
# from utils.dataset import DataSet
# from utils.generate_test_splits import kfold_split, get_stances_for_folds
# from utils.score import report_score, LABELS, score_submission

# from utils.system import parse_params, check_version
def generate_features_new(stances,dataset,name):
    h, b, y = [],[],[]

    for stance in stances:
        y.append(LABELS.index(stance['Stance']))
        h.append(stance['Headline'])
        b.append(dataset.articles[stance['Body ID']])

    return y
  

competition_dataset = DataSet("competition_test")
y_competition = generate_features_new(competition_dataset.stances, competition_dataset, "competition")

Xs = dict()
ys = dict()

# Load/Precompute all features now
#Run on competition dataset
actual = [LABELS[int(a)] for a in y_competition]
# predict_1= [LABELS[int(a)] for a in y_DT]


print("Scores on the test set")
# report_score(actual,predict_1)

Reading dataset
Total stances: 25413
Total bodies: 904
Scores on the test set


In [ ]:
d=DataSet()
h,b,y= generate_features(d.stances,d)

combined_list = [list(x) for x in zip(h, b,y)]

combine_df_train=pd.DataFrame(combined_list,columns = ["Headline","articleBody","Stances"])
combine_df_train.to_csv("./data/combined_list_1.csv")



Reading dataset
Total stances: 49972
Total bodies: 1683


In [ ]:
competition_dataset = DataSet("competition_test")
h_test,b_test,y_test= generate_features(competition_dataset.stances,competition_dataset)

combined_list_test = [list(x_test) for x_test in zip(h_test, b_test, y_test)]

combine_df_test=pd.DataFrame(combined_list_test,columns = ["Headline","articleBody","Stances"])
combine_df_test.to_csv("./data/combined_list_test.csv")



Reading dataset
Total stances: 25413
Total bodies: 904


In [ ]:
# Pre-processing involves removal of puctuations and converting text to lower case
word_seq_head_train = [text_to_word_sequence(head) for head in combine_df_train['Headline']]
word_seq_bodies_train = [text_to_word_sequence(body) for body in combine_df_train['articleBody']]
word_seq_head_test = [text_to_word_sequence(head) for head in combine_df_test['Headline']]
word_seq_bodies_test = [text_to_word_sequence(body) for body in combine_df_test['articleBody']]

In [ ]:
# Combine headline and body together
word_seq_train = [list(i) for i in word_seq_head_train]
for i in range(len(word_seq_head_train)):
    word_seq_train[i].extend(word_seq_bodies_train[i]) 
    
word_seq_test = [list(i) for i in word_seq_head_test]
for i in range(len(word_seq_head_test)):
    word_seq_test[i].extend(word_seq_bodies_test[i])

In [ ]:
fix_words = word_seq_train + word_seq_test

In [ ]:
fix = [" ".join(i) for i in fix_words]

In [ ]:
fix[10]

'gateway pundit a british rapper whose father is awaiting trial in manhattan for a pair of us embassy bombings is a leading suspect in the barbaric beheading of american journalist james foley it was revealed on friday abdel majed abdel bary — who recently tweeted a photo of himself holding up a severed head — was among three brits identified as possibly being the masked killer known as “john the beatle ” bary 24 is the son of an egyptian born militant who is awaiting trial on terror charges tied to the deadly 1998 bombings of embassies in kenya and tanzania also under investigation are the brother of a british doctor once charged with kidnapping two western war correspondents and a former gang member who converted to islam and traveled to syria britain’s telegraph newspaper reported a dozen american counterterrorism experts are expected to fly to the uk “within days” to help identify foley’s killer britain’s daily mail reported former hostages held by isis have said he is one of sever

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_all_tfidf = vectorizer.fit_transform(fix)

In [ ]:
X_train_tfidf = X_all_tfidf[0:len(word_seq_train), :]
X_test_tfidf = X_all_tfidf[len(word_seq_train): len(fix), :]

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

## DECISION TREE CLASSIFIER

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state=0)
tree.fit(X_train_tfidf, y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [ ]:
y_DT = tree.predict(X_test_tfidf)

In [ ]:
actual = [LABELS[int(a)] for a in y_competition]
predict_1= [LABELS[int(a)] for a in y_DT]


print("Scores on the test set")
report_score(actual,predict_1)

Scores on the test set
Score 4748.0
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    320    |    66     |    256    |   1261    |
-------------------------------------------------------------
| disagree  |    126    |    21     |    79     |    471    |
-------------------------------------------------------------
|  discuss  |    583    |    56     |   1180    |   2645    |
-------------------------------------------------------------
| unrelated |   2754    |    425    |   3428    |   11742   |
-------------------------------------------------------------
Score: 4748.0 out of 11651.25	(40.750992382791544%)


40.750992382791544

## XGBOOST

In [ ]:
import xgboost as xgb

model1 = xgb.XGBClassifier()

train_model1 = model1.fit(X_train_tfidf, y)



In [ ]:
model2 = xgb.XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5)

train_model2 = model2.fit(X_train_tfidf, y)


In [ ]:
pred1 = train_model1.predict(X_test_tfidf)
pred2 = train_model2.predict(X_test_tfidf)

In [ ]:
predict_1= [LABELS[int(a)] for a in pred1]
print("Scores on the test set")
report_score(actual,predict_1)

Scores on the test set
Score 4631.25
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |     0     |     0     |    17     |   1886    |
-------------------------------------------------------------
| disagree  |     0     |     0     |     1     |    696    |
-------------------------------------------------------------
|  discuss  |     0     |     0     |    96     |   4368    |
-------------------------------------------------------------
| unrelated |     1     |     0     |    225    |   18123   |
-------------------------------------------------------------
Score: 4631.25 out of 11651.25	(39.7489539748954%)


39.7489539748954

In [ ]:
predict_1= [LABELS[int(a)] for a in pred2]
print("Scores on the test set")
report_score(actual,predict_1)

Scores on the test set
Score 4767.25
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |     9     |     0     |    79     |   1815    |
-------------------------------------------------------------
| disagree  |     6     |     0     |    19     |    672    |
-------------------------------------------------------------
|  discuss  |     4     |     0     |    424    |   4036    |
-------------------------------------------------------------
| unrelated |    62     |     0     |   1058    |   17229   |
-------------------------------------------------------------
Score: 4767.25 out of 11651.25	(40.91621070700569%)


40.91621070700569

## RANDOM FOREST CLASSIFIER

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100, random_state=0)
forest.fit(X_train_tfidf, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
y_RF = forest.predict(X_test_tfidf)


In [ ]:
actual = [LABELS[int(a)] for a in y_competition]
predict_1= [LABELS[int(a)] for a in y_RF]

print("Scores on the test set")
report_score(actual,predict_1)

Scores on the test set
Score 4766.25
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |     2     |     0     |    92     |   1809    |
-------------------------------------------------------------
| disagree  |     0     |     0     |     3     |    694    |
-------------------------------------------------------------
|  discuss  |    11     |     0     |    451    |   4002    |
-------------------------------------------------------------
| unrelated |     0     |     0     |   1202    |   17147   |
-------------------------------------------------------------
Score: 4766.25 out of 11651.25	(40.90762793691664%)


40.90762793691664

### LINEAR SVM

In [ ]:
from sklearn.svm import LinearSVC
linear_svm = LinearSVC().fit(X_train_tfidf, y)

In [ ]:
y_SVM = linear_svm.predict(X_test_tfidf)

In [ ]:
actual = [LABELS[int(a)] for a in y_competition]
predict_1= [LABELS[int(a)] for a in y_SVM]

print("Scores on the test set")
report_score(actual,predict_1)

Scores on the test set
Score 5364.5
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    325    |     0     |   1346    |    232    |
-------------------------------------------------------------
| disagree  |    73     |     0     |    545    |    79     |
-------------------------------------------------------------
|  discuss  |    183    |     0     |   3746    |    535    |
-------------------------------------------------------------
| unrelated |   1998    |     0     |   13324   |   3027    |
-------------------------------------------------------------
Score: 5364.5 out of 11651.25	(46.04227014268855%)


46.04227014268855

## SVM

In [ ]:
from sklearn.svm import SVC
svm = SVC().fit(X_train_tfidf, y)

In [ ]:
y_svm = svm.predict(X_test_tfidf)

In [ ]:
actual = [LABELS[int(a)] for a in y_competition]
predict_1= [LABELS[int(a)] for a in y_svm]

print("Scores on the test set")
report_score(actual,predict_1)

Scores on the test set
Score 4813.75
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |     0     |     0     |    20     |   1883    |
-------------------------------------------------------------
| disagree  |     3     |     0     |     3     |    691    |
-------------------------------------------------------------
|  discuss  |     2     |     0     |    327    |   4135    |
-------------------------------------------------------------
| unrelated |     1     |     0     |    429    |   17919   |
-------------------------------------------------------------
Score: 4813.75 out of 11651.25	(41.315309516146336%)


41.315309516146336